In [1]:
#!/usr/bin/env python
"""
"""
import json
import logging
import re
import uuid
from collections import defaultdict
from datetime import datetime, timedelta
from pathlib import Path

import netCDF4
import numpy as np
import rasterio

from datacube import Datacube
from datacube.index.hl import Doc2Dataset
from datacube.utils import changes


LOG = logging.getLogger(__name__)

def print_dict(doc):
    print(json.dumps(doc, indent=4, sort_keys=True, cls=NumpySafeEncoder))

def find_datasets(path: Path):
    """
    """
    pattern = re.compile(r'(?P<barra_var>accum_prcp)\-(?P<barra_var_type>fc\-spec)\-(?P<barra_sampling_frequency>PT1H)\-(?P<barra_domain>BARRA_R)\-(?P<barra_version>v1)\-(?P<date>\d{8})T(?P<time>\d{4})Z\.sub.nc')
    datasets = defaultdict(dict)
    for ncfile in path.iterdir():
        match = pattern.search(str(ncfile))
        if match:
            barra_var, barra_var_type, barra_sampling_frequency, barra_domain, barra_version, date, hour = match.groups()
            dataset = barra_var + date + hour+ barra_domain + barra_version
            datasets[dataset][barra_var] = ncfile
    return datasets

def generate_product_defn():
    return {
        'name': 'accum_prcp',
        'metadata_type': 'eo',
        'metadata': {
            'product_type': 'barra_accum_prcp',
            'format' : { 'name': 'NetCDF'}
        },
        'storage': {
            'crs': 'GEOGCS["unknown",DATUM["unknown",SPHEROID["Sphere",6371229,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]]',
            'resolution': {
                'latitude': -0.1100000035809413773,
                'longitude': 0.1100000058540808734
            }
        },
        'description': 'BARRA Hourly precipitation accumulation',
        'measurements': [
            {
                'name':'accum_prcp',
                'dtype':'float64',
                'nodata': -1.073741824,
                'units':'1'
            }
        ]
    }

def generate_dataset_docs(dataset_name, dataset):
    """
    """
    sample_ncfile = dataset['accum_prcp']
    sample_ncfile_gdal = f'NETCDF:{sample_ncfile}:accum_prcp'
    creation_time = datetime.fromtimestamp(sample_ncfile.stat().st_mtime)
    geo_ref_points, spatial_ref = get_grid_spatial_projection(
        sample_ncfile_gdal)
    
    date = name[10:22]

    start_time = datetime.strptime(date, '%Y%m%d%H%M')
    end_time = start_time + timedelta(hours=1) - timedelta(microseconds=1)
    center_time = start_time + timedelta(minutes=30)
    docs = []
    for i in range(6):
        unique_ds_uri = f'{sample_ncfile.as_uri()}#{creation_time}#{start_time+timedelta(hours=i)}'
        doc = {
            'id': str(uuid.uuid5(uuid.NAMESPACE_URL, unique_ds_uri)),
            'product_type': 'barra_accum_prcp',
            'creation_dt': str(creation_time),
            'extent': {
                'from_dt': str(start_time+timedelta(hours=i)),
                'to_dt': str(end_time+timedelta(hours=i)),
                'center_dt': str(center_time+timedelta(hours=i)),
                'coord': to_lat_long_extent(geo_ref_points),
            },
            'format': {'name': 'NetCDF'},
            'grid_spatial': {
                'projection': {
                    'geo_ref_points': geo_ref_points,
                    'spatial_reference': spatial_ref,
                }
            },
            'image': {
                'bands': {
                    'accum_prcp': {
                        'path': '',
                        'layer': 'accum_prcp',
                    }
                }
            },
            'lineage': {'source_datasets': {}}
        }
        docs.append(('file:'+str(dataset['accum_prcp'])+'#part='+str(i),doc))
    return docs


def to_lat_long_extent(geo_ref_points):
    return {corner: {'lat': points['y'], 'lon': points['x']}
            for corner, points in geo_ref_points.items()}


def get_grid_spatial_projection(fname):
    with rasterio.open(fname, 'r') as img:
        left, bottom, right, top = img.bounds
        spatial_reference = str(str(getattr(img, 'crs_wkt', None) or img.crs.wkt))
        geo_ref_points = {
            'ul': {'x': left, 'y': top},
            'ur': {'x': right, 'y': top},
            'll': {'x': left, 'y': bottom},
            'lr': {'x': right, 'y': bottom},
        }
        return geo_ref_points, spatial_reference

In [5]:
dc = Datacube()
index = dc.index

product_def = generate_product_defn()
product = index.products.from_doc(product_def)
indexed_product = index.products.add(product)
print(indexed_product)

DatasetType(name='accum_prcp', id_=140)


In [6]:
path = Path('/home/david/Downloads/BARRA/')
datasets = find_datasets(path)
resolver = Doc2Dataset(index)

for name, dataset in datasets.items():
    docs = generate_dataset_docs(name, dataset)
    for doc in docs:
        dataset, err = resolver(doc[1], doc[0])
        try:
            indexed_dataset = index.datasets.add(dataset)
            print(indexed_dataset)
        except Exception as e:
            logging.error("Couldn't index %s/%s", path, name)
            logging.exception("Exception", e)


Dataset <id=9bec1213-ed93-5cbc-9e39-893ec90961b8 type=accum_prcp location=file:/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc#part=0>
Dataset <id=73da95eb-dbca-5502-8794-e92313f74f4e type=accum_prcp location=file:/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc#part=1>
Dataset <id=0f21eaa7-14e2-571e-aaac-5b7a860ca502 type=accum_prcp location=file:/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc#part=2>
Dataset <id=87bf409b-2a9c-5f42-909a-68d21915f455 type=accum_prcp location=file:/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc#part=3>
Dataset <id=63bb10bb-87ff-5fbb-8b8f-30740eb76eed type=accum_prcp location=file:/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc#part=4>
Dataset <id=fe400e4b-627e-5804-bca0-449f35069bf7 type=accum_prcp location=file:/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v